In [1]:
import numpy as np
import bundle
import time
import ray
from tqdm import tqdm

In [2]:
# ray.util.connect("192.168.10.68:8088")
ray.util.connect("192.168.100.108:32703")

{'num_clients': 1,
 'python_version': '3.8.5',
 'ray_version': '1.6.0',
 'ray_commit': '7916500c43de46721c51e6b95fb51cfa2c6078ba',
 'protocol_version': '2021-05-20'}

In [4]:
@ray.remote
def ray_bundles(idx):
    bundle_list, Z_list = ray.get(ray_env).create_bundles(idx,idx+1)
    return  bundle_list,Z_list
t1 = time.time()
env = bundle.Evaluator()
ray_env = ray.put(env)
return_refs = [ray_bundles.remote(idx) for idx in range(260, 265)]
return_items = ray.get(return_refs)
print(f"Total time = {time.time() - t1}")

init a new Data Manager here
Total time = 58.62239217758179


## update cluster environments and files

In [ ]:
import os
os.environ["KUBECONFIG"] = "./config/magics_cluster.yaml"
! ray up config/cluster.yaml -y

## submit job to cluster

In [ ]:
import os
os.environ["KUBECONFIG"] = "./config/magics_cluster.yaml"
! ray submit cluster.yaml scripts.py

## TIPS for control ray cluster
sync files:   ray rsync_up cluster.yaml 'bundle.py' '/home/ray/dev_ws/bundle.py'

update denpendency:  ray up cluster.yaml

connect through ssh:  ray attach cluster.yaml

execute command on head: ray exec cluster.yaml ""